# MPI codes

## jupyter tip: 
how could one see proper syntax?

```cpp

#include <stdio.h>
#include <mpi.h>
#include <string.h>

int main(int argc, char *argv[])
{
  int npes, myrank, namelen, i;
  char processor_name[MPI_MAX_PROCESSOR_NAME];
  char greeting[MPI_MAX_PROCESSOR_NAME + 80];
  MPI_Status status;
  
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &npes);
  MPI_Comm_rank(MPI_COMM_WORLD, &myrank);
  MPI_Get_processor_name( processor_name, &namelen );

  sprintf( greeting, "Hello, World, From process %d of %d on %s", myrank, npes, processor_name );

  if (myrank == 0 ) {
	  printf( "%s\n", greeting );
	  for (i = 1; i < npes; i++) {
		  MPI_Recv( greeting, sizeof( greeting ), MPI_CHAR, i, 1, MPI_COMM_WORLD, &status);
		  printf( "%s\n", greeting );
	   }
  }
  else {
	  MPI_Send( greeting, strlen( greeting ) + 1, MPI_CHAR, 0, 1, MPI_COMM_WORLD);
  }
    
  MPI_Finalize();
  return 0;
}

```

# BSend_Recv_1 C

In [11]:
%%writefile BSend_Recv_1.c

#include <stdio.h>
#include <mpi.h>
#include <string.h>

int main(int argc, char *argv[])
{
  int npes, myrank, namelen, i;
  char processor_name[MPI_MAX_PROCESSOR_NAME];
  char greeting[MPI_MAX_PROCESSOR_NAME + 80];
  MPI_Status status;
  
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &npes);
  MPI_Comm_rank(MPI_COMM_WORLD, &myrank);
  MPI_Get_processor_name( processor_name, &namelen );

  sprintf( greeting, "Hello, World, From process %d of %d on %s", myrank, npes, processor_name );

  if (myrank == 0 ) {
	  printf( "%s\n", greeting );
	  for (i = 1; i < npes; i++) {
		  MPI_Recv( greeting, sizeof( greeting ), MPI_CHAR, i, 1, MPI_COMM_WORLD, &status);
		  printf( "%s\n", greeting );
	   }
  }
  else {
	  MPI_Send( greeting, strlen( greeting ) + 1, MPI_CHAR, 0, 1, MPI_COMM_WORLD);
  }
    
  MPI_Finalize();
  return 0;
}

Overwriting BSend_Recv_1.c


## Compile

In [12]:
!mpicc -o BSend_Recv_1 BSend_Recv_1.c

## Run

In [13]:
!mpirun -n 2 BSend_Recv_1

Hello, World, From process 0 of 2 on morteza-TravelMate-P243
Hello, World, From process 1 of 2 on morteza-TravelMate-P243


#  BSend_Recv_1 Fortran

In [5]:
%%writefile BSend_Recv_1.for

       Program size_rank

         implicit none
         include 'mpif.h'

         INTEGER ierr,npes,myrank,namelen,i
         INTEGER stat(MPI_STATUS_SIZE)
         CHARACTER*(MPI_MAX_PROCESSOR_NAME) processor_name
         CHARACTER (MPI_MAX_PROCESSOR_NAME + 80) greeting
C         INTEGER len = MPI_MAX_PROCESSOR_NAME + 80
         CHARACTER(1) numb(0:9)
         
         numb(0)="0"
         numb(1)="1"
         numb(2)="2"
         numb(3)="3"
         numb(4)="4"
         numb(5)="5"
         numb(6)="6"
         numb(7)="7"
         numb(8)="8"
         numb(9)="9"
  
         call MPI_INIT( ierr )

         call MPI_COMM_SIZE( MPI_COMM_WORLD, npes, ierr )
         call MPI_COMM_RANK( MPI_COMM_WORLD, myrank, ierr )
         call MPI_GET_PROCESSOR_NAME(processor_name,namelen,ierr)
         
         greeting = 'Hello Wold , From process '//numb(myrank)//
     *' of '//numb(npes)//' on '//processor_name   
         IF(myrank.EQ.0) THEN
            print*, greeting 
            DO i = 1,  npes -1

              call MPI_RECV(greeting, len(greeting)+1,MPI_CHARACTER,i,1,
     *                   MPI_COMM_WORLD,stat,ierr)
              print*,greeting
           
            END DO
         ELSE
             call MPI_SEND(greeting,len(greeting)+1,MPI_CHARACTER,0,1,
     *            MPI_COMM_WORLD,ierr)
         ENDIF
         call MPI_FINALIZE(ierr)
       End   

Writing BSend_Recv_1.for


## Compile

In [6]:
!mpif90 -o BSend_Recv_1_fortran BSend_Recv_1.f

## Run

In [7]:
!mpirun -n 2 BSend_Recv_1_fortran

 Hello Wold , From process 0 of 2 on morteza-TravelMate-P243                                                                                                                                                                                                                                                                                    
 Hello Wold , From process 1 of 2 on morteza-TravelMate-P243                                                                                                                                                                                                                                                                                    


# BSend_Recv_2  CPP

In [8]:
%%writefile BSend_Recv_2.cpp

#include <stdio.h>
#include <mpi.h>
#include <iostream.h>
#define np 8

int main(int argc, char *argv[])
{
  int npes, myrank, i, j;
  int a[np+2][np+2];
  int left, right;
  MPI_Status stat;
  
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &npes);
  MPI_Comm_rank(MPI_COMM_WORLD, &myrank);
 
  for(j =0; j<= np+1; j++){
	  for(i=0; i<= np+1; i++)
		  a[i][j] = myrank*100+10*j+i;
  }
  
  left = myrank - 1;
  right= myrank + 1;
  
  if(myrank == 1 ) left = npes -1 ;
  if(myrank == (npes-1)) aright = 1;
  
if (myrank != 0 ) {
	  if(myrank%2==0){
		MPI_Send(&a[1][0],np+2,MPI_INT,left,1,MPI_COMM_WORLD);
		MPI_Send(&a[np][0],np+2,MPI_INT,right,1,MPI_COMM_WORLD);
		MPI_Recv(&a[0][0],np+2,MPI_INT,left,1,MPI_COMM_WORLD,&stat);
		MPI_Recv(&a[np+1][0],np+2,MPI_INT,right,1,MPI_COMM_WORLD,&stat);
	  }else{
		MPI_Recv(&a[np+1][0],np+2,MPI_INT,right,1,MPI_COMM_WORLD,&stat);
		MPI_Recv(&a[0][0],np+2,MPI_INT,left,1,MPI_COMM_WORLD,&stat);
 		MPI_Send(&a[np][0],np+2,MPI_INT,right,1,MPI_COMM_WORLD);
		MPI_Send(&a[1][0],np+2,MPI_INT,left,1,MPI_COMM_WORLD);
	  }
}

cout<<" This is node: "<<myrank<<'\n';
  for(j=0; j<=np+1; j++){
       for(i=0; i<=np+1; i++){
            cout<<a[i][j]<<" ";
       }
       cout<<'\n';       
  }

  MPI_Finalize();
  return 0;
}

Overwriting BSend_Recv_2.cpp


## Compile

## Run

# BSend_Recv_2 Fortran

In [33]:
%%writefile BSend_Recv_2.for

       Program size_rank

         implicit none
         include 'mpif.h'
         INTEGER np
         PARAMETER(np = 8)
         INTEGER ierr,npes,myrank,i,j
         INTEGER a(0:np+1,0:np+1)
         INTEGER left, right
         INTEGER stat(MPI_STATUS_SIZE)
           
         call MPI_INIT( ierr )
         call MPI_COMM_SIZE( MPI_COMM_WORLD, npes, ierr )
         call MPI_COMM_RANK( MPI_COMM_WORLD, myrank, ierr )

         DO j = 0, np+1
             DO i = 0, np+1
                a(i,j) = myrank*100+i*10+j
             END DO
         END DO

         left = myrank - 1
         right = myrank +1 
         IF(myrank.EQ.1) left = npes-1
         IF(myrank.EQ.(npes -1)) right = 1

         IF(myrank.NE.0) THEN
          IF(MOD(myrank,2).EQ.0)THEN
             call MPI_SEND(a(0,1),np+2,MPI_INTEGER,left,1,
     *            MPI_COMM_WORLD,ierr)
             call MPI_SEND(a(0,np),np+2,MPI_INTEGER,right,1,
     *            MPI_COMM_WORLD,ierr)
              call MPI_RECV(a(0,0),np+2,MPI_INTEGER,left,1,
     *                   MPI_COMM_WORLD,stat,ierr)
              call MPI_RECV(a(0,np+1),np+2,MPI_INTEGER,right,1,
     *                   MPI_COMM_WORLD,stat,ierr)
          ELSE
              call MPI_RECV(a(0,np+1),np+2,MPI_INTEGER,right,1,
     *                   MPI_COMM_WORLD,stat,ierr)
              call MPI_RECV(a(0,0),np+2,MPI_INTEGER,left,1,
     *                   MPI_COMM_WORLD,stat,ierr)
             call MPI_SEND(a(0,np),np+2,MPI_INTEGER,right,1,
     *            MPI_COMM_WORLD,ierr)
             call MPI_SEND(a(0,1),np+2,MPI_INTEGER,left,1,
     *            MPI_COMM_WORLD,ierr)
          ENDIF    
         END IF

         PRINT*,"This is node: ",myrank  
         DO i = 0,  np+1
              WRITE(*,*)(a(i,j), j=0,np+1)
         END DO
        call MPI_FINALIZE(ierr)
       End   

Writing BSend_Recv_2.for


## Compile

## Run

# main Cpp

In [10]:
%%writefile main.cpp


#include <iostream.h>
#include <mpi.h>

int main(int argc, char **argv)
{
  
  MPI_Init(&argc, &argv);
  
  cout<<"HI There\n";

  MPI_Finalize();
  return 0;
}


Overwriting main.cpp


# main Fortran

In [11]:
%%writefile main.for

       Program main

         implicit none
         include 'mpif.h'
         integer ierr,rc
         call MPI_INIT( ierr )

         print*,' HI There'

         call MPI_FINALIZE(rc)
       End   


Writing main.for


## Compile

## Run

# NBSend_Recv_1 Cpp

In [12]:
%%writefile omp_hello.for


#include <stdio.h>
#include <mpi.h>
#include <string.h>

int main(int argc, char *argv[])
{
  int npes, myrank, namelen, i;
  char processor_name[MPI_MAX_PROCESSOR_NAME];
  char greeting[MPI_MAX_PROCESSOR_NAME + 80];
  MPI_Status status;
  MPI_Request request;
  
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &npes);
  MPI_Comm_rank(MPI_COMM_WORLD, &myrank);
  MPI_Get_processor_name( processor_name, &namelen );

  sprintf( greeting, "Hello, World, From process %d of %d on %s", myrank, npes, processor_name );

  if (myrank == 0 ) {
	  printf( "%s\n", greeting );
	  for (i = 1; i < npes; i++) {
		  MPI_Irecv( greeting, sizeof( greeting ), MPI_CHAR, i, 1, MPI_COMM_WORLD, &request);
	//	 MPI_Wait(&request,&status);
		  printf( "%s\n", greeting );
		  
	   }
  }
  else {
	  MPI_Isend( greeting, strlen( greeting ) + 1, MPI_CHAR, 0, 1, MPI_COMM_WORLD,&request);
	  MPI_Wait(&request,&status);
  }
//   MPI_Waitall(2,request,status); 
  MPI_Finalize();
  return 0;
}


Writing omp_hello.for


## Compile

## Run

# NBSend_Recv_1 Fortran

In [13]:
%%writefile NBSend_Recv_1.for


       Program size_rank

         implicit none
         include 'mpif.h'

         INTEGER ierr,npes,myrank,namelen,i
         INTEGER stat(MPI_STATUS_SIZE)
         INTEGER request
         CHARACTER*(MPI_MAX_PROCESSOR_NAME) processor_name
         CHARACTER (MPI_MAX_PROCESSOR_NAME + 80) greeting
C         INTEGER len = MPI_MAX_PROCESSOR_NAME + 80
         CHARACTER(1) numb(0:9)
         
         numb(0)="0"
         numb(1)="1"
         numb(2)="2"
         numb(3)="3"
         numb(4)="4"
         numb(5)="5"
         numb(6)="6"
         numb(7)="7"
         numb(8)="8"
         numb(9)="9"
  
         call MPI_INIT( ierr )

         call MPI_COMM_SIZE( MPI_COMM_WORLD, npes, ierr )
         call MPI_COMM_RANK( MPI_COMM_WORLD, myrank, ierr )
         call MPI_GET_PROCESSOR_NAME(processor_name,namelen,ierr)
         
         greeting = 'Hello Wold , From process '//numb(myrank)//
     *' of '//numb(npes)//' on '//processor_name   
         IF(myrank.EQ.0) THEN
            print*, greeting 
            DO i = 1,  npes -1

             call MPI_IRECV(greeting, len(greeting)+1,MPI_CHARACTER,i,1,
     *                   MPI_COMM_WORLD,request,ierr)
            
             call MPI_WAIT(request,stat,ierr)
             print*,greeting
            END DO
         ELSE
            call MPI_ISEND(greeting,len(greeting)+1,MPI_CHARACTER,0,1,
     *            MPI_COMM_WORLD,request,ierr)
            call MPI_WAIT(request,stat,ierr)     
         ENDIF
         call MPI_FINALIZE(ierr)
       End   



Writing NBSend_Recv_1.for


## Compile

## Run

# NBSend_Recv_2 CPP


In [14]:
%%writefile NBSend_Recv_2.cpp


#include <stdio.h>
#include <mpi.h>
#include <string.h>
#include <iostream.h>
int main(int argc, char *argv[])
{
  int npes, myrank, namelen, i;
  char processor_name[MPI_MAX_PROCESSOR_NAME];
  char greeting[MPI_MAX_PROCESSOR_NAME + 80];
  int flag;
  MPI_Status status;
  MPI_Request request;
  
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &npes);
  MPI_Comm_rank(MPI_COMM_WORLD, &myrank);
  MPI_Get_processor_name( processor_name, &namelen );

  sprintf( greeting, "Hello, World, From process %d of %d on %s", myrank, npes, processor_name );

  if (myrank == 0 ) {
	  printf( "%s\n", greeting );
	  for (i = 1; i < npes; i++) {
		  MPI_Irecv( greeting, sizeof( greeting ), MPI_CHAR, i, 1, MPI_COMM_WORLD, &request);
		  MPI_Wait(&request,&status);
		  MPI_Test(&request,&flag,&status);
		  printf( "%s", greeting );
		  cout<<" &  flag is: "<<flag<<'\n';
	   }
  }
  else {
	  MPI_Isend( greeting, strlen( greeting ) + 1, MPI_CHAR, 0, 1, MPI_COMM_WORLD,&request);
	  MPI_Wait(&request,&status);
	  MPI_Test(&request,&flag,&status);
  }
//   MPI_Waitall(2,request,status); 
  MPI_Finalize();
  return 0;
}


Writing NBSend_Recv_2.cpp


## Compile

## Run

# NBSend_Recv_2 Fortran

In [16]:
%%writefile NBSend_Recv_2.for



       Program size_rank

         implicit none
         include 'mpif.h'

         INTEGER ierr,npes,myrank,namelen,i
         INTEGER stat(MPI_STATUS_SIZE)
         INTEGER request
         CHARACTER*(MPI_MAX_PROCESSOR_NAME) processor_name
         CHARACTER (MPI_MAX_PROCESSOR_NAME + 80) greeting
C         INTEGER len = MPI_MAX_PROCESSOR_NAME + 80
         CHARACTER(1) numb(0:9)
         LOGICAL flag 
         
         numb(0)="0"
         numb(1)="1"
         numb(2)="2"
         numb(3)="3"
         numb(4)="4"
         numb(5)="5"
         numb(6)="6"
         numb(7)="7"
         numb(8)="8"
         numb(9)="9"
  
         call MPI_INIT( ierr )

         call MPI_COMM_SIZE( MPI_COMM_WORLD, npes, ierr )
         call MPI_COMM_RANK( MPI_COMM_WORLD, myrank, ierr )
         call MPI_GET_PROCESSOR_NAME(processor_name,namelen,ierr)
         
         greeting = 'Hello Wold , From process '//numb(myrank)//
     *' of '//numb(npes)//' on '//processor_name   
         IF(myrank.EQ.0) THEN
            print*, greeting 
            DO i = 1,  npes -1

             call MPI_IRECV(greeting, len(greeting)+1,MPI_CHARACTER,i,1,
     *                   MPI_COMM_WORLD,request,ierr)
            
             call MPI_WAIT(request,stat,ierr)
             call MPI_TEST(request,flag,stat,ierr)
             print*,greeting," & flag is: ",flag
            END DO
         ELSE
            call MPI_ISEND(greeting,len(greeting)+1,MPI_CHARACTER,0,1,
     *            MPI_COMM_WORLD,request,ierr)
            call MPI_WAIT(request,stat,ierr)
            call MPI_TEST(request,flag,stat,ierr)     
         ENDIF
         call MPI_FINALIZE(ierr)
       End   

Overwriting NBSend_Recv_2.for


## Compile

## Run

# NBSend_Recv_3 CPP

In [17]:
%%writefile NBSend_Recv_3.cpp


#include <stdio.h>
#include <mpi.h>
#include <iostream.h>
#define np 8

int main(int argc, char *argv[])
{
  int npes, myrank, i, j;
  int a[np+2][np+2];
  int left, right;
  MPI_Status stat[4];
  MPI_Request request[4];
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &npes);
  MPI_Comm_rank(MPI_COMM_WORLD, &myrank);
 
  for(j =0; j<= np+1; j++){
	  for(i=0; i<= np+1; i++)
		  a[i][j] = myrank*100+10*j+i;
  }
  
  left = myrank - 1;
  right= myrank + 1;
  
  if(myrank == 1 ) left = npes -1 ;
  if(myrank == (npes-1)) right = 1;
  
if (myrank != 0 ) {
	//communication with left processor
	MPI_Isend(&a[1][0],np+2,MPI_INT,left,1,MPI_COMM_WORLD,&request[0]);
	MPI_Irecv(&a[0][0],np+2,MPI_INT,left,1,MPI_COMM_WORLD,&request[1]);
        //communication with right processor
	MPI_Isend(&a[np][0],np+2,MPI_INT,right,1,MPI_COMM_WORLD,&request[2]);
	MPI_Irecv(&a[np+1][0],np+2,MPI_INT,right,1,MPI_COMM_WORLD,&request[3]);
//        MPI_Waitall(4,request,stat);

	for(i = 0; i < npes; i++)
		MPI_Wait(&request[i],&stat[i]);
}

cout<<" This is node: "<<myrank<<'\n';
  for(j=0; j<=np+1; j++){
       for(i=0; i<=np+1; i++){
            cout<<a[i][j]<<" ";
       }
       cout<<'\n';       
  }

  MPI_Finalize();
  return 0;
}

Writing NBSend_Recv_3.cpp


## Compile

## Run

# NBSend_Recv_3 Fortran

In [27]:
%%writefile NBSend_Recv_3.for


       Program size_rank

         implicit none
         include 'mpif.h'
         INTEGER np
         PARAMETER(np = 8)
         INTEGER ierr,npes,myrank,i,j
         INTEGER a(0:np+1,0:np+1)
         INTEGER left, right
         INTEGER request(4) 
         INTEGER stat(MPI_STATUS_SIZE,4)
           
         call MPI_INIT( ierr )
         call MPI_COMM_SIZE( MPI_COMM_WORLD, npes, ierr )
         call MPI_COMM_RANK( MPI_COMM_WORLD, myrank, ierr )

         DO j = 0, np+1
             DO i = 0, np+1
                a(i,j) = myrank*100+i*10+j
             END DO
         END DO

         left = myrank - 1
         right = myrank +1 
         IF(myrank.EQ.1) left = npes-1
         IF(myrank.EQ.(npes -1)) right = 1

         IF(myrank.NE.0) THEN
c communication with left processor

       call MPI_ISEND(a(0,1),np+2,MPI_INTEGER,left,1,
     *            MPI_COMM_WORLD,request(1),ierr)
       call MPI_IRECV(a(0,0),np+2,MPI_INTEGER,left,1,
     *                   MPI_COMM_WORLD,request(2),ierr)
c communication with right processor
       call MPI_ISEND(a(0,np),np+2,MPI_INTEGER,right,1,
     *            MPI_COMM_WORLD,request(3),ierr)
       call MPI_IRECV(a(0,np+1),np+2,MPI_INTEGER,right,1,
     *                   MPI_COMM_WORLD,request(4),ierr)
       call MPI_WAITALL(4,request,stat,ierr) 
         END IF

         PRINT*,"This is node: ",myrank  
         DO i = 0,  np+1
              WRITE(*,*)(a(i,j), j=0,np+1)
         END DO
        call MPI_FINALIZE(ierr)
       End   



Overwriting NBSend_Recv_3.for


## Compile

## Run

# size_rank Cpp

In [29]:
%%writefile size_rank.cpp

#include <iostream.h>
#include <mpi.h>

int main(int argc, char **argv)
{
  int npes, myrank;
  MPI_Init(&argc, &argv);

  MPI_Comm_size(MPI_COMM_WORLD, &npes);
  MPI_Comm_rank(MPI_COMM_WORLD, &myrank);

  cout<<"HI There, I am node "<<myrank<<"  and the total worker"
	  <<" which you are using now is: "<<npes<<'\n';

  MPI_Finalize();
  return 0;
}


Writing size_rank.cpp


## Complie

## Run

# size_rank Fortran

In [22]:
%%writefile size_rank.for

       Program size_rank

         implicit none
         include 'mpif.h'
         integer ierr,npes,myrank
         call MPI_INIT( ierr )

         call MPI_COMM_SIZE( MPI_COMM_WORLD, npes, ierr )
         call MPI_COMM_RANK( MPI_COMM_WORLD, myrank, ierr )

         print*,"HI There, I am node ",myrank," and the total",
     *"number of workers which you are using now is: ",npes

         call MPI_FINALIZE(ierr)
       End

Writing size_rank.for


## Compile

## Run

# Extra C codes

## Model.cpp

In [35]:
%%writefile Model.cpp

#include <stdio.h>
#include <mpi.h>
#include <iostream.h>
#define npp 8

int main(int argc, char *argv[])
{
  int npes, myrank, i, j, k;
  int nx, ny;
  int left, right;
  MPI_Status statn,stat[4];
  MPI_Request request[4];
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &npes);
  MPI_Comm_rank(MPI_COMM_WORLD, &myrank);

  nx = npp/(npes-1);
  ny = npp;
  int a[nx+2][ny+2];

  for(j =0; j<= ny+1; j++){
	  for(i=0; i<= nx+1; i++)
		  a[i][j] = myrank*100+10*j+i;
  }
  
  left = myrank - 1;
  right= myrank + 1;
  
  if(myrank == 1 ) left = npes -1 ;
  if(myrank == (npes-1)) right = 1;
  
if (myrank != 0 ) {

	 for(i=0; i<=nx+1; i++){
		a[i][0] = a[i][ny];
		a[i][ny+1]=a[i][1];
	}




	//communication with left processor
	MPI_Isend(&a[1][0],ny+2,MPI_INT,left,1,MPI_COMM_WORLD,&request[0]);
	MPI_Irecv(&a[0][0],ny+2,MPI_INT,left,1,MPI_COMM_WORLD,&request[1]);
        //communication with right processor
	MPI_Isend(&a[nx][0],ny+2,MPI_INT,right,1,MPI_COMM_WORLD,&request[2]);
	MPI_Irecv(&a[nx+1][0],ny+2,MPI_INT,right,1,MPI_COMM_WORLD,&request[3]);
        MPI_Waitall(4,request,stat);

//	for(i = 0; i < npes; i++)
//		MPI_Wait(&request[i],&stat[i]);

		MPI_Send(&a[0][0],(nx+2)*(ny+2),MPI_INT,0,1,MPI_COMM_WORLD);
}
if(myrank==0){  
	for(k=1; k<=npes-1; k++){
		MPI_Recv(&a[0][0],(nx+2)*(ny+2),MPI_INT,k,1,MPI_COMM_WORLD,&statn);
		cout<<" this is node: "<<myrank<<" I've received data from node:  "<<k<<'\n';
		  for(j=0; j<=ny+1; j++){
		       for(i=0; i<=nx+1; i++){
		            cout<<a[i][j]<<" ";
		       }
		       cout<<'\n';       
		  }
	}//k
}//if

  MPI_Finalize();
  return 0;
}


Writing Model.cpp


## Compile

## Run

##  Para_Temp.cpp

In [38]:
%%writefile Para_Temp.cpp


#include <stdio.h>
#include <iostream.h>
#include <fstream.h>
#include <math.h>
#include <string.h>
#include <mpi.h>                
                  //--------<the system geometry>-----------------
                  //    (0,0),,,,,,,,,,BC,,,,,,,,(0,n+1)
                  //      , (1,1) ...........(1,n) ,
                  //      ,   .                .   ,
                  //      B   .                .   B
                  //      C   .                .   C
                  //      ,   .                .   ,
                  //      ,   .                .   ,
                  //      , (n,1)............(n,n) ,
                  //  (n+1,0),,,,,,,,,BC,,,,,,,(n+1,n+1) 
                  //----------------------------------------------
#define n_tot 64

int main(int argc, char *argv[])
{
  int i, j, k, rk, nx, ny;
  double t, dt, s, t_max;
  char snp[20];
  int npes, myrank;
  int left, right;
  MPI_Status statn, stat[4];
  MPI_Request request[4];

  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &npes);
  MPI_Comm_rank(MPI_COMM_WORLD, &myrank);
 
  nx = n_tot/(npes-1);
  ny = n_tot;
  double Told[nx+2][ny+2],Tnew[nx+2][ny+2],temp[n_tot+2][n_tot+2];

//  if(myrank==0){
	  sprintf(snp, "%s", "result/snapP.dat");
	  ofstream outfile_snp(snp);
//  }

  left = myrank - 1;
  right= myrank + 1;
  
  if(myrank == 1 ) left = npes -1 ;
  if(myrank == (npes-1)) right = 1;
 
  t_max = 100;
  dt = 0.05;
  s = dt;


  for(j=0; j<=ny+1; j++)              //----------------------------------------  
	for(i=0; i<=nx+1; i++)        // ----------<initial condition>----------
		Tnew[i][j] = 0.;      //----------------------------------------  
if(myrank==1) Tnew[2][ny/2+1]=100;          //----------------------------------------SHOULD BE CORRECT



  t = 0.; 
  k = 0;

  while(t<=t_max)
  {

  if(myrank!=0){
	for (j=1; j<=ny; j++){
		for(i=1; i<=nx; i++){
			Told[i][j] = Tnew[i][j];
		}
	}

	


//---------------------------------------<Boandary Conditions Exchanges>---------------------------------
	for(i=0; i<=nx+1; i++){ 
		Told[i][0] = Tnew[i][ny];
		Told[i][ny+1]=Tnew[i][1]; 
	}

	//communication with left processor
	MPI_Isend(&Tnew[1][0],ny+2,MPI_DOUBLE,left,1,MPI_COMM_WORLD,&request[0]);
	MPI_Irecv(&Told[0][0],ny+2,MPI_DOUBLE,left,1,MPI_COMM_WORLD,&request[1]);
        //communication with right processor
	MPI_Isend(&Tnew[nx][0],ny+2,MPI_DOUBLE,right,1,MPI_COMM_WORLD,&request[2]);
	MPI_Irecv(&Told[nx+1][0],ny+2,MPI_DOUBLE,right,1,MPI_COMM_WORLD,&request[3]);
        MPI_Waitall(4,request,stat);
//------------------------------------------------------------------------------------------------------

	for (j=1; j<=ny; j++){
		for(i=1; i<=nx; i++){
			Tnew[i][j]=s*(Told[i][j+1]+Told[i][j-1]+Told[i+1][j]+Told[i-1][j])+(1.-4.*s)*Told[i][j];
		}
	}
  }//parallel if

	t += dt;
	k++;

	if(k%100==0){
		if(myrank!=0){
			MPI_Send(&Tnew[0][0],(nx+2)*(ny+2),MPI_DOUBLE,0,1,MPI_COMM_WORLD);
		}else{
			for(rk=1; rk<=npes-1; rk++){
				MPI_Recv(&Tnew[0][0],(nx+2)*(ny+2),MPI_DOUBLE,rk,1,MPI_COMM_WORLD,&statn);

				for(j=1; j<=ny; j++)
					for(i=1; i<=nx; i++)
						temp[nx*(rk-1)+i][j]=Tnew[i][j];
			}
				
			for (j=1; j<=n_tot; j++){
				for(i=1; i<=n_tot; i++){
					outfile_snp<<i<<"  "<<j<<"  "<<temp[i][j]<<endl;
				}
			}
			
		}

	}


  }//while

  MPI_Finalize();


}


Writing Para_Temp.cpp


## compile

### Run

## temp.cpp

In [40]:
%%writefile temp.cpp

#include <stdio.h>
#include <iostream.h>
#include <fstream.h>
#include <math.h>
#include <string.h>
                 
                  //--------<the system geometry>-----------------
                  //    (0,0),,,,,,,,,,BC,,,,,,,,(0,n+1)
                  //      , (1,1) ...........(1,n) ,
                  //      ,   .                .   ,
                  //      B   .                .   B
                  //      C   .                .   C
                  //      ,   .                .   ,
                  //      ,   .                .   ,
                  //      , (n,1)............(n,n) ,
                  //  (n+1,0),,,,,,,,,BC,,,,,,,(n+1,n+1) 
                  //----------------------------------------------
#define n_tot 64

int main(int argc, char *argv[])
{
  int i, j, k, nx, ny;
  nx = n_tot;
  ny = n_tot;
  double Told[nx+2][ny+2],Tnew[nx+2][ny+2];
  double t, dt, s, t_max;
  char snp[20];


  sprintf(snp, "%s", "result/snap.dat");
  ofstream outfile_snp(snp);

  t_max = 100;
  dt = 0.05;
  s = dt;


  for(j=0; j<=ny+1; j++)              //----------------------------------------  
	for(i=0; i<=nx+1; i++)        // ----------<initial condition>----------
		Tnew[i][j] = 0.;      //----------------------------------------  
  Tnew[nx/2+1][2]=100;           //----------------------------------------

  t = 0.; 
  k = 0;

  while(t<=t_max)
  {
	for (j=1; j<=ny; j++){
		for(i=1; i<=nx; i++){
			Told[i][j] = Tnew[i][j];
		}
	}

	
	for(j=0; j<=ny+1; j++){
		Told[0][j] = Tnew[nx][j];    //---------------------------------------
		Told[nx+1][j]=Tnew[1][j];    //---------------------------------------
	}			             //----------<Boandary Conditions>--------
	for(i=0; i<=nx+1; i++){ 	     //---------------------------------------
		Told[i][0] = Tnew[i][ny];    //---------------------------------------
		Told[i][ny+1]=Tnew[i][1];    //---------------------------------------
	}

	for (j=1; j<=ny; j++){
		for(i=1; i<=nx; i++){
			Tnew[i][j]=s*(Told[i][j+1]+Told[i][j-1]+Told[i+1][j]+Told[i-1][j])+(1.-4.*s)*Told[i][j];
		}
	}

	t += dt;
	k++;

	if(k%100==0){
		for (j=1; j<=ny; j++){
			for(i=1; i<=nx; i++){
				outfile_snp<<i<<"  "<<j<<"  "<<Tnew[i][j]<<endl;
			}
		}

	}	
  }//while
cout<<k<<endl;
}


Writing temp.cpp


## Compile

## Run